# Определение стоимости автомобилей

В задании нужно построить модель для определения стоимости автомобиля по ряду признаков. 

Необходимо учитывать:
- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

Необходимо загрузить и обработать данные из файла ***/datasets/autos.csv***. Известны признаки:

- DateCrawled — дата скачивания анкеты из базы
- VehicleType — тип автомобильного кузова
- RegistrationYear — год регистрации автомобиля
- Gearbox — тип коробки передач
- Power — мощность (л. с.)
- Model — модель автомобиля
- Kilometer — пробег (км)
- RegistrationMonth — месяц регистрации автомобиля
- FuelType — тип топлива
- Brand — марка автомобиля
- NotRepaired — была машина в ремонте или нет
- DateCreated — дата создания анкеты
- NumberOfPictures — количество фотографий автомобиля
- PostalCode — почтовый индекс владельца анкеты (пользователя)
- LastSeen — дата последней активности пользователя


Целевой признак:

- Price — цена (евро)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

import catboost as cb
import lightgbm as lgb
from lightgbm import LGBMRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore")

In [ ]:
data = pd.read_csv('/datasets/autos.csv')
data.info()
data.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [ ]:
data['DateCrawled'].max(), data['DateCrawled'].min()

('2016-04-07 14:36:58', '2016-03-05 14:06:22')

In [ ]:
data['DateCreated'].max(), data['DateCreated'].min()

('2016-04-07 00:00:00', '2014-03-10 00:00:00')

In [ ]:
data['LastSeen'].max(), data['LastSeen'].min()

('2016-04-07 14:58:51', '2016-03-05 14:15:08')

In [ ]:
data['Power'].max(), data['Power'].min()

(20000, 0)

In [ ]:
len(data['Power'].unique())

712

In [ ]:
sorted(data['Power'].unique())

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [ ]:
len(data['PostalCode'].unique())

8143

In [ ]:
sorted(data['RegistrationYear'].unique())

[1000,
 1001,
 1039,
 1111,
 1200,
 1234,
 1253,
 1255,
 1300,
 1400,
 1500,
 1600,
 1602,
 1688,
 1800,
 1910,
 1915,
 1919,
 1920,
 1923,
 1925,
 1927,
 1928,
 1929,
 1930,
 1931,
 1932,
 1933,
 1934,
 1935,
 1936,
 1937,
 1938,
 1940,
 1941,
 1942,
 1943,
 1944,
 1945,
 1946,
 1947,
 1948,
 1949,
 1950,
 1951,
 1952,
 1953,
 1954,
 1955,
 1956,
 1957,
 1958,
 1959,
 1960,
 1961,
 1962,
 1963,
 1964,
 1965,
 1966,
 1967,
 1968,
 1969,
 1970,
 1971,
 1972,
 1973,
 1974,
 1975,
 1976,
 1977,
 1978,
 1979,
 1980,
 1981,
 1982,
 1983,
 1984,
 1985,
 1986,
 1987,
 1988,
 1989,
 1990,
 1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2066,
 2200,
 2222,
 2290,
 2500,
 2800,
 2900,
 3000,
 3200,
 3500,
 3700,
 3800,
 4000,
 4100,
 4500,
 4800,
 5000,
 5300,
 5555,
 5600,
 5900,
 5911,
 6000,
 6500,
 7000,
 7100,
 7500,
 7800,
 8000,
 8200,

При рассмотрении показателя "почтовый индекс" видно что параметры или некорректны или в неизвестной кодировке (попадаются индексы Германии, Кореи и тд). Принимаю решение исключить этот параметр.


При рассмотрении таких параметров как дата скачивания анкеты из базы и дата регистрации анкеты видно что все анкеты скачаны в период апрель-май 2016 года а зарегистрированы в период с 2014 по 2016 год. 
При рассмотрении даты постановки автомобиля на учёт видно много очевидно некорректных значений, от 1000 и 1910 годов до 2066 и 9999 годов.
Принимаю решение отсортировать данные по правдоподобным значениям даты регистрации (с 1950 по 2016) и в дальнейшем рассматривать только их. Так же необходимо посчитать возраст автомобиля - вычесть дату регистрации из даты создания анкеты. Этот параметр может заменить и дату регистрации и дату создания анкеты, поэтому их из рассмотрения исключу.

При рассмотрении параметра "мощность" так же обнаружил множество некорректных данных. Оставляю только строки в которых мощьность от 50 до 500 лс.

Параметры "дата активности" показывают похожие результаты в 2016 году. Этот параметр мало влияет на цену, можно удалить.

In [ ]:
data = data[(data['Power'] > 50) & (data['Power'] < 500)]

In [ ]:
data = data[(data['RegistrationYear'] > 1950) & (data['RegistrationYear'] <= 2016)]

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 292677 entries, 1 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        292677 non-null  object
 1   Price              292677 non-null  int64 
 2   VehicleType        281380 non-null  object
 3   RegistrationYear   292677 non-null  int64 
 4   Gearbox            287516 non-null  object
 5   Power              292677 non-null  int64 
 6   Model              281519 non-null  object
 7   Kilometer          292677 non-null  int64 
 8   RegistrationMonth  292677 non-null  int64 
 9   FuelType           277043 non-null  object
 10  Brand              292677 non-null  object
 11  NotRepaired        249895 non-null  object
 12  DateCreated        292677 non-null  object
 13  NumberOfPictures   292677 non-null  int64 
 14  PostalCode         292677 non-null  int64 
 15  LastSeen           292677 non-null  object
dtypes: int64(7), object(

In [ ]:
def age(str):
    return int(str['DateCreated'].split('-')[0]) - str['RegistrationYear']

In [ ]:
data['age'] = data.apply(age, axis=1)

In [ ]:
data['age']

1          5
2         12
3         15
4          8
5         21
          ..
354361     0
354362    12
354366    16
354367    20
354368    14
Name: age, Length: 292677, dtype: int64

In [ ]:
data = data.drop(['DateCrawled', 'RegistrationYear', 'RegistrationMonth', 'PostalCode', 'DateCreated', 'LastSeen'], axis=1)

In [ ]:
sorted(data['NumberOfPictures'].unique())

[0]

In [ ]:
data = data.drop('NumberOfPictures', axis=1)

Данные в столбце 'NumberOfPictures' состоят из одних нулей, видимо в результате ошибки выгрузки. Удаляю этот столбец.

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 292677 entries, 1 to 354368
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Price        292677 non-null  int64 
 1   VehicleType  281380 non-null  object
 2   Gearbox      287516 non-null  object
 3   Power        292677 non-null  int64 
 4   Model        281519 non-null  object
 5   Kilometer    292677 non-null  int64 
 6   FuelType     277043 non-null  object
 7   Brand        292677 non-null  object
 8   NotRepaired  249895 non-null  object
 9   age          292677 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 24.6+ MB


In [ ]:
sum(data.duplicated())

38372

In [ ]:
data = data.drop_duplicates()

In [ ]:
data

,Price,VehicleType,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired,age
1,18300,coupe,manual,190,NaN,125000,gasoline,audi,yes,5
2,9800,suv,auto,163,grand,125000,gasoline,jeep,NaN,12
3,1500,small,manual,75,golf,150000,petrol,volkswagen,no,15
4,3600,small,manual,69,fabia,90000,gasoline,skoda,no,8
5,650,sedan,manual,102,3er,150000,petrol,bmw,yes,21
...,...,...,...,...,...,...,...,...,...,...
354361,5250,NaN,auto,150,159,150000,NaN,alfa_romeo,no,0
354362,3200,sedan,manual,225,leon,150000,petrol,seat,yes,12
354366,1199,convertible,auto,101,fortwo,125000,petrol,smart,no,16
354367,9200,bus,manual,102,transporter,150000,gasoline,volkswagen,no,20


Таблица выглядит правдоподобно. Дубликаты удалены. После предобработки мы потеряли (1 - 254305/354369 = 0,28) или 28% данных.
Выделю столбцы с категориальными данными.

In [ ]:
cat = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']

Необходимо решить что делать с пропусками. Можно заметить что все пропуска расположены в столбцах с категориальными значениями. Можно заменить их например значением 'empty' чтобы модель обрабатывала их как отдельное значение.


In [ ]:
for i in cat:
    data.loc[data[i].isna(), [i]] = 'empty'

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 254305 entries, 1 to 354368
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Price        254305 non-null  int64 
 1   VehicleType  254305 non-null  object
 2   Gearbox      254305 non-null  object
 3   Power        254305 non-null  int64 
 4   Model        254305 non-null  object
 5   Kilometer    254305 non-null  int64 
 6   FuelType     254305 non-null  object
 7   Brand        254305 non-null  object
 8   NotRepaired  254305 non-null  object
 9   age          254305 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 21.3+ MB


Обработку категориальных признаков выполняем двумя способами. 

1 - для единичных моделей (линейн. регрессии и решающего дерева) использую One Hot Encoding. 

2 - для моделей градиентного бустинка переведу столбцы с категориальными данными в тип 'category'.
В дальнейшем буду использовать два набора данных data_lin и data_grad_bust.

In [ ]:
data_lin = data.copy(deep=True)
data_grad_bust = data.copy(deep=True)

In [ ]:
data_lin = pd.get_dummies(data_lin, drop_first = True)

In [ ]:
for col in cat:
    data_grad_bust[col] = data[col].astype('category')

Создам функцию которая делит данные на признаки и целевой признак, тренировочную и тестовую выборки.

In [ ]:
def split(dat):
    X_train, X_test, Y_train, Y_test = train_test_split(dat.drop(['Price'], axis=1), dat['Price'],
                                                        test_size=0.2, random_state=12345)
    return X_train, X_test, Y_train, Y_test

# Вывод

Загрузил данные. Убрал неправдоподобные значения. Обработал пропуски. Обработал категориальные данные и подготовил функцию для разбиения на выборки. После предобработки потеряно 28% данных.

## Обучение моделей

Необходимо обучить несколько моделей с перебором гиперпараметров.

In [ ]:
X_train_l, X_test_l, Y_train_l, Y_test_l = split(data_lin)

In [ ]:
#ti_1 = time.perf_counter()
#model_l_r = LinearRegression()
#model_l_r.fit(X_train, Y_train)
#test_preds = model_l_r.predict(X_test)
#RMSE_l_r = mean_squared_error(Y_test, test_preds, squared= False) 
#time_l_r = (time.perf_counter() - ti_1) / 60
#print(f'RMSE линейной регрессии:{RMSE_l_r},   время работы линейной регрессии:  {time_l_r} мин')

#СТАРЫЙ ВАРИАНТ

In [ ]:
ti_1 = time.perf_counter()
model_l_r = LinearRegression()
cross_l_r = cross_val_score(model_l_r, X_train_l, Y_train_l, scoring = 'neg_root_mean_squared_error', cv=4)

RMSE_l_r = abs(cross_l_r.mean())
time_l_r = (time.perf_counter() - ti_1) / 60
print(f'RMSE линейной регрессии:{RMSE_l_r},   время работы линейной регрессии:  {time_l_r} мин')
model_l_r.fit(X_train_l, Y_train_l)

RMSE линейной регрессии:2668.3920151729026,   время работы линейной регрессии:  2.0879321007834126 мин


LinearRegression()

In [ ]:
data_tree = data_grad_bust.copy(deep=True)
for i in cat:
    data_tree[i] = data_tree[i].cat.codes
X_train_t, X_test_t, Y_train_t, Y_test_t = split(data_tree)

Для решающего дерева использую label encoding.

In [ ]:
ti_2 = time.perf_counter()
d_t_r = DecisionTreeRegressor()
parametrs = { 'max_depth': range (4, 13, 2),
              'min_samples_leaf': range (1, 8),
              'min_samples_split': range (2, 10, 2),
            }
grid_d_t_r = GridSearchCV(d_t_r, parametrs, scoring = 'neg_root_mean_squared_error', cv=4)
grid_d_t_r.fit(X_train_t, Y_train_t)
RMSE_d_t_r = abs(grid_d_t_r.best_score_)
time_d_t_r = (time.perf_counter() - ti_2) / 60
print(f'RMSE решающего дерева:{RMSE_d_t_r},   время работы решающего дерева:  {time_d_t_r} мин')

RMSE решающего дерева:2018.6040699499076,   время работы решающего дерева:  2.9877987089566886 мин


In [ ]:
X_train, X_test, Y_train, Y_test = split(data_grad_bust)

In [ ]:
ti_3 = time.perf_counter()

cb_r = cb.CatBoostRegressor(loss_function='RMSE', iterations=75, verbose=25)
params = {'depth': [7, 10, 14], 'learning_rate': [0.1, 0.3, 0.6]}
cb_grid = GridSearchCV(cb_r, params, cv = 4, scoring = 'neg_root_mean_squared_error') 
cb_grid.fit(X_train, Y_train, cat_features = cat)
RMSE_cb = abs(cb_grid.best_score_)
time_cb = (time.perf_counter() - ti_3) / 60
print(f'RMSE CatBoost:{RMSE_cb},   время работы CatBoost:  {time_cb} мин')


Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 4378.6364699	total: 99.5ms	remaining: 7.37s
25:	learn: 2097.4439095	total: 2.5s	remaining: 4.72s
50:	learn: 1925.3786573	total: 4.95s	remaining: 2.33s
74:	learn: 1860.5410810	total: 6.98s	remaining: 0us
0:	learn: 4380.0712046	total: 99.9ms	remaining: 7.39s
25:	learn: 2087.1404836	total: 2.18s	remaining: 4.11s
50:	learn: 1910.6522269	total: 4.12s	remaining: 1.94s
74:	learn: 1845.1261524	total: 6.04s	remaining: 0us
0:	learn: 4387.3381087	total: 95.1ms	remaining: 7.04s
25:	learn: 2100.3352742	total: 2.74s	remaining: 5.16s
50:	learn: 1923.2578075	total: 4.77s	remaining: 2.25s
74:	learn: 1867.6347920	total: 6.62s	remaining: 0us
0:	learn: 4379.3972788	total: 82.2ms	remaining: 6.08s
25:	learn: 2098.5783662	total: 2.17s	remaining: 4.08s
50:	learn: 1924.2355286	total: 4.39s	remaining: 2.06s
74:	learn: 1867.4939705	total: 6.3s	remaining: 0us
0:	learn: 3814.2981641	total: 80.8ms	remaining: 5.98s
25:	learn: 1863.3319655	total: 2.21s	remaining: 4.16s
50:	learn: 1780.8450587	total: 4.29s	r

In [ ]:
ti_4 = time.perf_counter()

lgb = LGBMRegressor(n_estimators=75)
params = {'max_depth': [7, 10, 14], 'learning_rate': [0.1, 0.3, 0.6]}
lgb_grid = GridSearchCV(lgb, params, cv = 4, scoring = 'neg_root_mean_squared_error')
lgb_grid.fit(X_train, Y_train, categorical_feature = cat)
RMSE_lgb = abs(lgb_grid.best_score_)
time_lgb = (time.perf_counter() - ti_4) / 60
print(f'RMSE LightGBM:{RMSE_lgb},   время работы LightGBM:  {time_lgb} мин')

RMSE LightGBM:1732.838326144851,   время работы LightGBM:  154.05073803208458 мин


# Вывод

Обучил несколько моделей и простые (линейную регрессию и решающее дерево) и работающие по принципу градиентного бустинга. 

## Анализ моделей

Нужно проанализировать полученные результаты.

In [ ]:
df_rez = pd.DataFrame([['Линейная регрессия', RMSE_l_r, time_l_r],
                  ['Решающее дерево', RMSE_d_t_r, time_d_t_r],
                  ['CatBoost', RMSE_cb, time_cb],     
                  ['LightGBM',RMSE_lgb, time_lgb]], 
columns=['Модель ML','RMSE обучения', 'Время обучения'])

In [ ]:
df_rez[['RMSE обучения', 'Время обучения']] = df_rez[['RMSE обучения', 'Время обучения']].astype('int')
df_rez

,Модель ML,RMSE обучения,Время обучения
0,Линейная регрессия,2668,2
1,Решающее дерево,2018,2
2,CatBoost,1730,11
3,LightGBM,1732,154


Составил таблицу скорости обучения с перебором параметров и RMSE на обучающей выборке.

In [ ]:
ti_1_test = time.perf_counter()
test_preds_l_r = model_l_r.predict(X_test_l)
RMSE_l_r_test = mean_squared_error(Y_test_l, test_preds_l_r, squared= False) 
time_l_r_test = (time.perf_counter() - ti_1_test) / 60

ti_2_test = time.perf_counter()
test_preds_d_t_r = grid_d_t_r.predict(X_test_t)
RMSE_d_t_r_test = mean_squared_error(Y_test_t, test_preds_d_t_r, squared= False) 
time_d_t_r_test = (time.perf_counter() - ti_2_test) / 60

ti_3_test = time.perf_counter()
test_preds_cb = cb_grid.predict(X_test)
RMSE_cb_test = mean_squared_error(Y_test, test_preds_cb, squared= False)
time_cb_test = (time.perf_counter() - ti_3_test) / 60

ti_4_test = time.perf_counter()
test_preds_lgb = lgb_grid.predict(X_test)
RMSE_lgb_test = mean_squared_error(Y_test, test_preds_lgb, squared= False)
time_lgb_test = (time.perf_counter() - ti_4_test) / 60

In [ ]:
df_test = pd.DataFrame([['Линейная регрессия', RMSE_l_r_test, time_l_r_test],
                  ['Решающее дерево', RMSE_d_t_r_test, time_d_t_r_test],
                  ['CatBoost', RMSE_cb_test, time_cb_test],     
                  ['LightGBM',RMSE_lgb_test, time_lgb_test]], 
columns=['Модель ML','RMSE проверки', 'Время проверки'])
df_test

,Модель ML,RMSE проверки,Время проверки
0,Линейная регрессия,2641.925523,0.004490
1,Решающее дерево,1994.676246,0.000300
2,CatBoost,1690.842217,0.001155
3,LightGBM,1717.971249,0.006408


Составил таблицу скорости работы обученной модели и RMSE на тестовой выборке.

In [ ]:
f'Среднее значение целевой переменной в тестовой выборке (средняя цена автомобиля): {round (Y_test.mean(), 1)}'

'Среднее значение целевой переменной в тестовой выборке (средняя цена автомобиля): 4945.8'

# Вывод

Свёл метрики и время для разных моделей  в таблицу. Для того чтобы было на что ориентироваться вывел среднее значение цены автомобиля.

# Общий вывод проекта
 
По результатам работы нескольких моделей можно сделать вывод что самая быстрая - линейная регрессия. Перебор параметров в её случае не требуется. При этом она даёт вполне приемлемое представление о ценах. Решающее дерево за счёт перебора параметров даёт немного лучший результат но времени требуется намного больше. При этом данные модели требуют обработки категориальных признаков.
Модель CatBoost в свою очередь требует меньше времени и даёт лучшие результаты чем решающее дерево. LightGBM требует очень много времени а  результаты показывает почти такие же как у CatBoost. Для этих моделей обработка категориальных признаков происходит сильно проще.

После подбора гиперпараметров и обучения предсказания у всех моделей занимают примерно одинаковое время различаясь на доли секунды.

В то же время необходимо отметить что данные результаты могут быть не вполне достоверными из-за моего пока ещё слабого понимания какие параметры и в каких рамках нужно перебирать. Ещё результаты сравнения могут сильно зависеть от данных.

Рекомендация:
После сравнения всех моделей лучшие результаты для задачи предсказания цены автомобиля по имеющимся данным показала модель CatBoost. Она точнее простых единичных моделей и гораздо быстрее LightGBM.